In [7]:
import os

In [9]:
os.chdir("../")

In [11]:
os.getcwd()

'/home/ajana/Code/MLProjects/Heart-Disease-Detection'

In [29]:
from heart.components.data_ingesion import DataIngestion
from heart.entity.artifact_entity import DataIngestionArtifact
from heart.entity.config_entity import TrainingPipelineConfig,DataIngestionConfig





In [30]:
training_config = TrainingPipelineConfig()

In [31]:
data_config = DataIngestionConfig(training_pipeline_config=training_config)

In [32]:
ingestion = DataIngestion(data_config)

In [33]:
ingestion.read_data_from_feature_store()

In [38]:
dframe = pd.read_csv(data_config.feature_store_file_path)

In [42]:
dframe.columns

Index(['age', 'gender', 'impluse', 'pressurehight', 'pressurelow', 'glucose',
       'kcm', 'troponin', 'class'],
      dtype='object')

In [44]:
df = pd.read_csv("/home/ajana/Code/MLProjects/Heart-Disease-Detection/artifact/12_25_2023_19_03_57/data_ingestion/feature_store/Heart Attack.csv")

In [46]:
df.columns()

TypeError: 'Index' object is not callable